In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from HFCS_bmimPF6_PR import configuration

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m2 = ConcreteModel()

m2.fs = FlowsheetBlock(default={"dynamic": False})

m2.fs.properties = GenericParameterBlock(default=configuration)

m2.fs.F101 = Flash(default={"property_package": m2.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

2021-04-30 12:35:26 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2021-04-30 12:35:26 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [3]:
m1.fs.properties.PR_kappa['R32', 'R125'].fix(-0.002535)
m1.fs.properties.PR_kappa['R125', 'R32'].fix(-0.002535)
m1.fs.properties.PR_kappa['R32', 'bmimPF6'].fix(-0.01435)
m1.fs.properties.PR_kappa['bmimPF6', 'R32'].fix(0.9651)
m1.fs.properties.PR_kappa['bmimPF6', 'R125'].fix(0.6312)
m1.fs.properties.PR_kappa['R125', 'bmimPF6'].fix(0.0504)

m2.fs.properties.PR_kappa['R32', 'R125'].fix(-0.002535)
m2.fs.properties.PR_kappa['R125', 'R32'].fix(-0.002535)
m2.fs.properties.PR_kappa['R32', 'bmimPF6'].fix(-0.05355)
m2.fs.properties.PR_kappa['bmimPF6', 'R32'].fix(0.1818)
m2.fs.properties.PR_kappa['bmimPF6', 'R125'].fix(1.6358)
m2.fs.properties.PR_kappa['R125', 'bmimPF6'].fix(0.04800)

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Set pressures to loop over
T = [286,301,310,320,330,340,350,360,370,380,390,400]

# Set fraction of feed compositions of HFCs to loop over
HFC_feed_frac = [0.001,0.15,0.2,0.3,0.4,0.5,0.72,0.91,0.99]

# Make outlet arrays
IL_x = np.zeros((len(T),len(HFC_feed_frac)))
IL_y = np.zeros((len(T),len(HFC_feed_frac)))
R32_x = np.zeros((len(T),len(HFC_feed_frac)))
R32_y = np.zeros((len(T),len(HFC_feed_frac)))
R125_x = np.zeros((len(T),len(HFC_feed_frac)))
R125_y = np.zeros((len(T),len(HFC_feed_frac))) 

for i in range(len(T)):
    
    # Fix pressure
    m1.fs.F101.inlet.temperature.fix(T[i])
    
    for j in range(len(HFC_feed_frac)):
        
        # Fix inlet mole fractions
        fract = 1 - HFC_feed_frac[j] 
        m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.3*HFC_feed_frac[j])
        m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1*fract)
        m1.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.7)   

        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)
        # model.display()
        
        # If solution is optimal store the concentration, and calculated temperatures in the created arrays
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i,j] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"])
            IL_y[i,j] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "bmimPF6"])
            R32_x[i,j] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i,j] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i,j] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i,j] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            #Change this to feasible
            print("Running temperature:",T[i],"and HFC feed fraction",HFC_feed_frac[j])
#             print(value(m1.fs.F101.liq_outlet.flow_mol[0]))
#             print(value(m1.fs.F101.vap_outlet.flow_mol[0]))
        else:
            print('Infeasible.')

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.
Infeasible.
Running temperature: 286 and HFC feed fraction 0.15
Running temperature: 286 and HFC feed fraction 0.2
Running temperature: 286 and HFC feed fraction 0.3
    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.
Infeasible.
Running temperature: 286 and HFC feed fraction 0.5
Running temperature: 286 and HFC feed fraction 0.72
Running temperature: 286 and HFC feed fraction 0.91
Running temperature: 286 and HFC feed fraction 0.99


ValueError: Cannot load a SolverResults object with bad status: error